In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_percentage_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
import matplotlib.pyplot as plt

# Load the data
tesla_data = pd.read_csv('Tesla_Close_2019_2024.csv', parse_dates=['Date'], index_col='Date')
sp500_data = pd.read_csv('S&P500_Close_2019_2024.csv', parse_dates=['Date'], index_col='Date')
ixic_data = pd.read_csv('IXIC_Close_2019_2024.csv', parse_dates=['Date'], index_col='Date')

# Merge datasets on 'Date'
merged_data = tesla_data.merge(sp500_data, left_index=True, right_index=True, suffixes=('', '_sp500'))
merged_data = merged_data.merge(ixic_data, left_index=True, right_index=True, suffixes=('', '_ixic'))

# Preprocess the data
data = merged_data[['Close', 'Close_sp500', 'Close_ixic']]
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(data)

# Create the dataset function
def create_dataset(dataset, time_step=60):
    X, y = [], []
    for i in range(time_step, len(dataset)):
        X.append(dataset[i-time_step:i])
        y.append(dataset[i, 0])  # Predict the 'Close' price
    return np.array(X), np.array(y)

# Create training and testing datasets
time_step = 60
train_size = int(len(scaled_data) * 0.8)
train_data = scaled_data[:train_size]
test_data = scaled_data[train_size - time_step:]

X_train, y_train = create_dataset(train_data, time_step)
X_test, y_test = create_dataset(test_data, time_step)

# Build the LSTM model with optimized hyperparameters
model = Sequential([
    LSTM(units=100, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])),
    Dropout(0.2),
    LSTM(units=50, return_sequences=False),
    Dropout(0.2),
    Dense(1)
])

model.compile(optimizer='adam', loss='mean_squared_error')
model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test))

# Make predictions
train_predictions = model.predict(X_train)
test_predictions = model.predict(X_test)

# Rescale predictions back to original scale
train_predictions_rescaled = scaler.inverse_transform(
    np.hstack([train_predictions, X_train[:, -1, 1:]])
)[:, 0]
test_predictions_rescaled = scaler.inverse_transform(
    np.hstack([test_predictions, X_test[:, -1, 1:]])
)[:, 0]

# Calculate RMPE
def rmpe(y_true, y_pred):
    return np.sqrt(mean_absolute_percentage_error(y_true, y_pred))

train_rmpe = rmpe(merged_data['Close'].values[time_step:train_size], train_predictions_rescaled)
test_rmpe = rmpe(merged_data['Close'].values[train_size:], test_predictions_rescaled)
print(f"Training RMPE: {train_rmpe:.4f}")
print(f"Testing RMPE: {test_rmpe:.4f}")

# Visualize actual vs. predicted
plt.figure(figsize=(14, 6))
plt.plot(merged_data.index, merged_data['Close'], label='Actual Data')
plt.plot(merged_data.index[time_step:train_size], train_predictions_rescaled, label='Predicted Training Data')
plt.plot(merged_data.index[train_size:], test_predictions_rescaled, label='Predicted Test Data')
plt.title('Actual vs. Predicted Tesla Stock Prices')
plt.xlabel('Date')
plt.ylabel('Stock Price (USD)')
plt.legend()
plt.show()


ModuleNotFoundError: No module named 'tensorflow'

In [ ]:
p